In [2]:
"""
This notebook is used to create a table for the ground truth data.

Overall, we have 20 pairs of train/test dataset. Each train/test pair will have two 
corresponding feature networks and label networks. The training and testing data
have been generated and saved in ./outputs/16_dgl_csv/

We use the csv files to create a table for the paper. 

"""

'\nThis notebook is used to create a table for the ground truth data.\n\nOverall, we have 20 pairs of train/test dataset. Each train/test pair will have two \ncorresponding feature networks and label networks. The training and testing data\nhave been generated and saved in ./outputs/16_dgl_csv/\n\nWe use the csv files to create a table for the paper. \n\n'

In [2]:
import csv
import pandas as pd
from collections import Counter

In [3]:
path = "../../outputs/16_dgl_csv/1977_"   

In [4]:
## first, we retrieve the training and testing data for the clf
# number of positive links in the training data, number of sampled negative links in the training data
# number of positive links in the testing data, number of negative links in the testing data
train_pos,train_neg,test_pos,test_neg = list(),list(),list(),list()

for t in range(2002,2022):
    
    edges_df = pd.read_csv(path+str(t)+"/clf/edges.csv")
    train_df, test_df = edges_df[edges_df["train_mask"] == True],edges_df[edges_df["train_mask"] == False]
    train_dict, test_dict = Counter(train_df["label"]),Counter(test_df["label"])
    train_pos.append(train_dict.get(1))
    train_neg.append(train_dict.get(0))
    test_pos.append(test_dict.get(1))
    test_neg.append(test_dict.get(0))
    

In [5]:
## next, we retrieve the metadata used to build the training and testing networks
# .  i.e., the feature networks and the label networks
# number of link use in the feature network of the training data, number of link use in the feature network of the testing data, 
# number of fixed nodes in each train/test pair
train_link,test_link,nodes = list(),list(),list()

for t in range(2002,2022):
    # get the link use of feature networks of training and testing data
    network_df = pd.read_csv(path+str(t)+"/ne/edges.csv")
    train_fea_df, test_fea_df = network_df[network_df["train_fea_mask"] == True],network_df[network_df["test_fea_mask"] == True]
    train_fea_link, test_fea_link = train_fea_df.shape[0],test_fea_df.shape[0]
    # display(network_df)

    # get the fixed node numbers for each train/test dataset 
    node_df = pd.read_csv(path+str(t)+"/ne/nodes.csv")
    fixed_nodes = node_df.shape[0]

    train_link.append(train_fea_link)
    test_link.append(test_fea_link)
    nodes.append(fixed_nodes)
    # break




In [6]:
## create a dataframe for the training and testing data that train a classifier
year_idx = [t for t in range(2002,2022)]
data = ["train_data","test_data"]
cols = pd.MultiIndex.from_product([data,["pos_links","neg_links"]])
train_clf = pd.DataFrame(
    zip(train_pos,train_neg,test_pos,test_neg),
    columns=cols,
    index=year_idx
)

In [7]:
train_clf

train_data           test_data          
      pos_links neg_links pos_links neg_links
2002       4309      4309      4238    169198
2003      12661     12661      9511   1512606
2004      11868     11868     12060   3503711
2005      13082     13082     14309   5345284
2006      15214     15214     14169   7506135
2007      14957     14957      5117  10219633
2008       5289      5289     15584  13199906
2009      15810     15810     16195  14026886
2010      16979     16979     14741  17210827
2011      15413     15413     11725  21073898
2012      12206     12206     18473  25399895
2013      18924     18924     16618  28876787
2014      17129     17129     18627  33725997
2015      19164     19164     16030  38654749
2016      16720     16720     14356  45046804
2017      14744     14744     11675  50308893
2018      11884     11884     14228  54657681
2019      14355     14355      8012  58097447
2020       8045      8045      7740  61653324
2021       7820      7820       430  64034468

In [8]:
print(train_clf.to_latex(index=True))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{train\_data} & \multicolumn{2}{l}{test\_data} \\
{} &  pos\_links & neg\_links & pos\_links & neg\_links \\
\midrule
2002 &       4309 &      4309 &      4238 &    169198 \\
2003 &      12661 &     12661 &      9511 &   1512606 \\
2004 &      11868 &     11868 &     12060 &   3503711 \\
2005 &      13082 &     13082 &     14309 &   5345284 \\
2006 &      15214 &     15214 &     14169 &   7506135 \\
2007 &      14957 &     14957 &      5117 &  10219633 \\
2008 &       5289 &      5289 &     15584 &  13199906 \\
2009 &      15810 &     15810 &     16195 &  14026886 \\
2010 &      16979 &     16979 &     14741 &  17210827 \\
2011 &      15413 &     15413 &     11725 &  21073898 \\
2012 &      12206 &     12206 &     18473 &  25399895 \\
2013 &      18924 &     18924 &     16618 &  28876787 \\
2014 &      17129 &     17129 &     18627 &  33725997 \\
2015 &      19164 &     19164 &     16030 &  38654749 \\
2016 &      16720 &     16720

In [9]:
def get_density(nodes,links):
    """get density for a network,
        ns: nodes, ls: links, both are lists"""
    density = list()
    len_nodes = len(nodes)
    # train_link

    for i in range(len_nodes):
        num_nodes = nodes[i]
        num_links = links[i]
        d = 2*num_links/(num_nodes*num_nodes)
        density.append(str(round(d*100,4))+"%")
    
    return density

In [10]:
## create a dataframe for the training and testing networks

networks = ["training network"]*6+["testing network"]*6
fea_label = ["feature network"]*3+["label network"]*3+["feature network"]*3+["label network"]*3
range_use = ["time range", "link use", "density"]+["time range", "link use", "density"]+["time range", "link use", "density"]+["time range", "link use", "density"]
# net_cols = pd.MultiIndex.from_product([networks,fea_label,range_use])
net_cols = pd.MultiIndex.from_arrays([networks,fea_label,range_use])

year_idx = [t for t in range(2002,2022)]
tr_fea_idx = ["1977-"+str(t) for t in range(2000,2020)]
tr_lab_idx = [t for t in range(2001,2021)]
tt_fea_idx = ["1977-"+str(t) for t in range(2001,2021)]
tt_lab_idx = [t for t in range(2002,2022)]

train_density = get_density(nodes,train_link)
train_lab_density = get_density(nodes,train_pos)
test_density = get_density(nodes,test_link)
test_lab_density = get_density(nodes,test_pos)

network_data = pd.DataFrame(
    zip(tr_fea_idx,train_link,train_density,tr_lab_idx,train_pos,train_lab_density,tt_fea_idx,test_link,test_density,tt_lab_idx,test_pos,test_lab_density),
    columns=net_cols,
    index=nodes
)

network_data.rename_axis("Nodes",inplace=True)

In [11]:
network_data

training network                                                    \
       feature network                   label network                     
            time range link use  density    time range link use  density   
Nodes                                                                      
607          1977-2000     6176  3.3524%          2001     4309   2.339%   
1767         1977-2001    25483  1.6323%          2002    12661   0.811%   
2675         1977-2002    48836   1.365%          2003    11868  0.3317%   
3299         1977-2003    67376  1.2381%          2004    13082  0.2404%   
3905         1977-2004    87042  1.1416%          2005    15214  0.1995%   
4550         1977-2005   109268  1.0556%          2006    14957  0.1445%   
5168         1977-2006   130749  0.9791%          2007     5289  0.0396%   
5329         1977-2007   137565  0.9688%          2008    15810  0.1113%   
5900         1977-2008   159503  0.9164%          2009    16979  0.0976%   
6525         1977-2009   183514  0.8621%          2010    15413  0.0724%   
7161         1977-2010   205806  0.8027%          2011    12206  0.0476%   
7634         1977-2011   222832  0.7647%          2012    18924  0.0649%   
8248         1977-2012   248875  0.7317%          2013    17129  0.0504%   
8828         1977-2013   272435  0.6991%          2014    19164  0.0492%   
9527         1977-2014   299221  0.6593%          2015    16720  0.0368%   
10066        1977-2015   321833  0.6353%          2016    14744  0.0291%   
10491        1977-2016   341502  0.6206%          2017    11884  0.0216%   
10815        1977-2017   356891  0.6103%          2018    14355  0.0245%   
11140        1977-2018   375121  0.6045%          2019     8045   0.013%   
11352        1977-2019   385558  0.5984%          2020     7820  0.0121%   

      testing network                                                    
      feature network                   label network                    
           time range link use  density    time range link use  density  
Nodes                                                                    
607         1977-2001    10485  5.6914%          2002     4238  2.3005%  
1767        1977-2002    38144  2.4433%          2003     9511  0.6092%  
2675        1977-2003    60704  1.6967%          2004    12060  0.3371%  
3299        1977-2004    80458  1.4785%          2005    14309   0.263%  
3905        1977-2005   102256  1.3411%          2006    14169  0.1858%  
4550        1977-2006   124225  1.2001%          2007     5117  0.0494%  
5168        1977-2007   136038  1.0187%          2008    15584  0.1167%  
5329        1977-2008   153375  1.0802%          2009    16195  0.1141%  
5900        1977-2009   176482   1.014%          2010    14741  0.0847%  
6525        1977-2010   198927  0.9345%          2011    11725  0.0551%  
7161        1977-2011   218012  0.8503%          2012    18473   0.072%  
7634        1977-2012   241756  0.8297%          2013    16618   0.057%  
8248        1977-2013   266004   0.782%          2014    18627  0.0548%  
8828        1977-2014   291599  0.7483%          2015    16030  0.0411%  
9527        1977-2015   315941  0.6962%          2016    14356  0.0316%  
10066       1977-2016   336577  0.6644%          2017    11675   0.023%  
10491       1977-2017   353386  0.6422%          2018    14228  0.0259%  
10815       1977-2018   371246  0.6348%          2019     8012  0.0137%  
11140       1977-2019   383166  0.6175%          2020     7740  0.0125%  
11352       1977-2020   393378  0.6105%          2021      430  0.0007%

In [12]:
network_data.iloc[[-1]]

training network                                                    \
       feature network                   label network                     
            time range link use  density    time range link use  density   
Nodes                                                                      
11352        1977-2019   385558  0.5984%          2020     7820  0.0121%   

      testing network                                                    
      feature network                   label network                    
           time range link use  density    time range link use  density  
Nodes                                                                    
11352       1977-2020   393378  0.6105%          2021      430  0.0007%

In [13]:
print(network_data.to_latex(index=True))

\begin{tabular}{llrlrrllrlrrl}
\toprule
{} & \multicolumn{6}{l}{training network} & \multicolumn{6}{l}{testing network} \\
{} & \multicolumn{3}{l}{feature network} & \multicolumn{3}{l}{label network} & \multicolumn{3}{l}{feature network} & \multicolumn{3}{l}{label network} \\
{} &       time range & link use &  density &    time range & link use &  density &      time range & link use &  density &    time range & link use &  density \\
Nodes &                  &          &          &               &          &          &                 &          &          &               &          &          \\
\midrule
607   &        1977-2000 &     6176 &  3.3524\% &          2001 &     4309 &   2.339\% &       1977-2001 &    10485 &  5.6914\% &          2002 &     4238 &  2.3005\% \\
1767  &        1977-2001 &    25483 &  1.6323\% &          2002 &    12661 &   0.811\% &       1977-2002 &    38144 &  2.4433\% &          2003 &     9511 &  0.6092\% \\
2675  &        1977-2002 &    48836 &   1.365